EXERCISE 2: In this exercise I redo the Monte Carlo Simulation of Berry(1994). We have 500 simulated duopoly markets. The utility of each consumer in each market is given by 
$u_{ij} = \beta_0 + \beta_xx_j + \sigma_d\xi_j - \alpha p_j + \epsilon_{ij}$
with $\epsilon_{ij}$ follow T1EV. The utility of outside good is given by $u_{i0} = \epsilon_{i0}$ where $\epsilon_{i0}$ follow the same distribution. Marginal cost is constraint to be positive and is given by 
$c_j = e^{\gamma_0 + \gamma_xx_j+\sigma_c\xi_j + \sigma\xi_j + \gamma_ww_j + \sigma_\omega\omega_j }$

The firm maximizes the profit function $\pi_j = p_jx_j - C_j(x_j)$. The first-order condition is
$$
\frac{d\pi_j}{dp_j} = x_j + \frac{dx_j}{dp_j}p_j - \frac{dx_j}{dp_j}c_j = 0 \rightarrow p_j = c_j - \frac{x_j}{\frac{dx_j}{dp_j}}
$$
$$
p_j = c_j - \frac{s_j}{\frac{ds_j}{dp_j}}
$$
The FOC in the logit model is 
$$
p_j = c_j + \frac{1}{\alpha(1-s_j)}
$$
% See https://github.com/johnjosephhorton/berry1994/blob/master/berry.R for R code of this practice.

% See https://github.com/rkackerman/Industrial-Organization-HW-Example-One for MATLAB code of this pratice. This code has some conceptual mistakes but the way that he codes is very helpful.

In [49]:
# See the below link for Distribution
# https://juliastats.github.io/Distributions.jl/stable/starting/
using LinearAlgebra
using Random, Distributions
using Optim
using NLsolve
using DataFrames

In [98]:
# Parameters

J = 2 #Number of products
N = 100 #Number of individuals
M = 500 #Number of markets
Jchar = 1 #Number of product characteristics
iter = 100 #Number of iterations
β_0 = 5
β_x = 2
σ_d = 1 # or
# σ_d = 3
α = 1
γ_o = 1
γ_x = .5
γ_w = .25
σ_w = .25
σ_c = .25
Y = zeros(2*M, 1)
X1 = zeros(2*M , 3)
β = zeros(3, iter)

3×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

# Equilibrium Price Functions

In [131]:
function equilibriumPrice_temp(price , Variables, ϵ)
    # price is equilibrium variable
    # I used FOC for the logit model
    X = Variables[1,:] 
    ξ = Variables[2,:]
    w = Variables[3,:]
    ω = Variables[4,:]
    u = [β_0 .+ β_x*X + σ_d*ξ - α*price'; 0] .+ ϵ
    u_max = maximum(u, dims = 1)
    choice_mat = (u_max .<= u)
    s = sum( choice_mat, dims = 2)/N
    c = exp.(γ_o .+ γ_x*X + σ_c*ξ + γ_w*w + σ_w*ω) # marginal cost (observed)
    equ = price' - c - 1 ./ ( α*(1 .- s[1:2]) )
    return equ
end

equilibriumPrice_temp (generic function with 2 methods)

# Data Generating Process Function

In [132]:
# Data Generating Process dgp
# This function is used to generate the observed data
# Observed data: p (prices), X(characteristics), c(marginal cost), s(market share), w(input characteristics)
function dgp(i)
    # Variables
    d1 = Normal()
    Random.seed!(100 + i) #Setting the seed
    Variables = rand(d1, 4, J)
    X = Variables[1,:] 
    ξ = Variables[2,:]
    w = Variables[3,:]
    ω = Variables[4,:]
    d2 = Gumbel()
    Random.seed!(200 + i) #Setting the seed
    ϵ = rand(d2 , J+1 , N) # error term
    equilibriumPrice(price) = equilibriumPrice_temp(price, Variables, ϵ )
    # Find the equilibrium prices
    initial_p = [.1  .1]
    solution = nlsolve( equilibriumPrice, initial_p)
    p = solution.zero'
    u = [β_0 .+ β_x*X + σ_d*ξ - α*p; 0] .+ ϵ
    u_max = maximum(u, dims = 1)
    choice_mat = (u_max .<= u)
    s = sum( choice_mat, dims = 2)/N
    c = exp.(γ_o .+ γ_x*X + σ_c*ξ + γ_w*w + σ_w*ω)
    return X, w, p, c, s
end

dgp (generic function with 1 method)

# OLS Function

In [125]:
function OLS(Y , X)
    
    #if rank( X'*X ) == 3
        β = ( (X'*X)^(-1) ) * X'*Y
        return β
    #end
    #return [0 ; 0 ; 0]
#    var_β = ...
end

OLS (generic function with 1 method)

First I regress $\delta_j = ln(s_j) - ln(s_0)$ on $x_j$ and $p_j$ without regard to the endogeneity of prices.

In [134]:
Y = zeros(2*M, 1)
X1 = zeros(2*M , 3)

for m in 1:2
    X, w, p , c, s = dgp(m)
    y = log.(s[1:2]) .- log( s[3] )
    x1 = [ones(J) X p ]
    Y[2*m-1:2*m,:] = y
    X1[2*m-1:2*m,:] = x1
end
#β
#    if rank( X1 ) == 3
#        β[:, counter] = OLS(Y , X1)
#        counter = counter + 1
#    end
#    i = i + 1
#end
#sum(β , dims = 2)/iter

In [135]:
X1

1000×3 Array{Float64,2}:
 1.0  -0.176283   7.09404
 1.0   1.09946    4.91987
 1.0  -0.0645184  2.62269
 1.0   0.660184   5.94125
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 ⋮                       
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    
 0.0   0.0        0.0    

In [118]:
OLS(Y, X1)

3×1 Array{Float64,2}:
  1.6905616365712077
  0.822960432767249 
 -0.4168404155969046

In [8]:
#= βhat = zeros(Jchar, iter)
for i in 1:iter
    
# Computing Share of Products
        X = dgp(i)
        δ = X*β
        d_2 = Gumbel() # or we can use d = Normal()
        Random.seed!(101 + i) #Setting the seed
        ϵ = rand(d_2, J , N)
        u = δ .+ ϵ
        u_max, ind = findmax(u, dims = 1)
        choice_mat = (u .>= u_max)
        s = sum( choice_mat, dims = 2)/N
    # Maximum Likelihood Estimation
        β_0 = ones(Jchar)
        optimum = optimize(loglikelihood, β_0)
        MLE = optimum.minimum
        βhat[ :, i] = optimum.minimizer
end
βhat
=#

UndefVarError: UndefVarError: X not defined

In [9]:
println( mean( βhat , dims = 2 ) )
println( var( βhat , dims = 2) )

[0.0]
[0.0]
